# Predict Workout

Predict Workout Day and distance

In [ ]:
#!pip install sklearn

In [ ]:
from utils import *

# import api_key from myconfig file
from myconfig import *

In [ ]:
# If you would like to start with clean data use below commented line
#import numpy as np
#data = loadCleanData()
## Define new coloumns
#data[['temp','wind','weather','day','hour','dayOfWeek','isWeekend']] = np.nan
#data[['weatherDesc']] = ""

data = loadCorolatedData()
data.head()

### Feature Extraction

In [ ]:
# Since data is limited these features doesn't make sense
#data['year'] = list(map(lambda t : t.year, data['Activity Date']))
#data['month'] = list(map(lambda t : t.month, data['Activity Date']))
#data['minute'] = list(map(lambda t : t.minute, data['Activity Date']))

In [ ]:
data['day'] = list(map(lambda t : t.day, data['Activity Date']))

In [ ]:
data['hour'] = list(map(lambda t : t.hour, data['Activity Date']))

In [ ]:
data['dayOfWeek'] = list(map(lambda t : t.dayofweek, data['Activity Date']))

In [ ]:
data['isWeekend'] = ((pd.DatetimeIndex(data['Activity Date']).dayofweek) // 5 == 1).astype(int)

#### Weather Data Export

In [ ]:
# api_key is defined in myconfig file
city = "Istanbul" 
date = "2020-12-25"
hour = 8

temp_data = get_weather(api_key,city,date,hour)
print(temp_data)

In [ ]:
date = "2021-03-05"
hour = 8
temp_data = get_weather(api_key,city,date,hour)
print(temp_data)

#### Get Weather Data and Save 

In [ ]:
## iterate for all ride date to extract weather
#for index, row in data.iterrows():
#    try:
#        temp, wind, weatherDesc = get_weather(api_key,city,row['Activity Date'],row['hour'])
#        data['temp'][index] = temp
#        data['wind'][index] = wind 
#        data['weatherDesc'][index] = weatherDesc
#        print(row['Activity Date'],temp, wind, weatherDesc)
#    except Exception as e:
#        print("Error:", e, row['Activity Date'])

### Transform Activity Type and Weather Description

In [ ]:
# convert string data to numeric value for decision making
from sklearn import preprocessing
le_activity = preprocessing.LabelEncoder()

le_activity.fit(data['Activity Type'])
print(le_activity.classes_)

vals = le_activity.transform(le_activity.classes_)
vals

# convert all ride type data into numeric values 
data['rideType'] = le_activity.transform(data['Activity Type'])
data['rideType']

In [ ]:
# Transform weather conditions data to numeric value for decision making
le = preprocessing.LabelEncoder()
le.fit(data['weatherDesc'])
print(le.classes_)

weather = le.transform(le.classes_)
print(weather)

# convert all weatherDesc data into numeric values
data['weather'] = le.transform(data['weatherDesc'])
data['weather']

In [ ]:
## Export corolated data
data.to_csv('data/activities_corolated.csv', index=False)
print('Successfully saved!')

### Prepare features

In [ ]:
# ride distribution per hour
data.plot(kind='scatter', x='hour', y='Distance', xticks=data['hour'], figsize=(14,8))

In [ ]:
# ride distribution per day
data.plot(kind='scatter', x='dayOfWeek', y='Distance', xticks=data['dayOfWeek'], figsize=(14,8))

In [ ]:
# ride distribution per weekdays
data.plot(kind='scatter', x='isWeekend', y='Distance', xticks=data['isWeekend'], figsize=(14,8))

In [ ]:
# indoor outdoor ride distribution per temp
data.plot(kind='scatter', x='rideType', y='temp', xticks=data['rideType'], figsize=(14,8))

In [ ]:
# Select features as list of array
X = data[['hour','dayOfWeek','isWeekend','temp','wind','weather']]
X = X.to_numpy()
X

### Prepare Predict

In [ ]:
Y_distance = data['Distance']
Y_distance = Y_distance.to_numpy()
Y_distance

In [ ]:
Y_rideType = data['rideType']
Y_rideType = Y_rideType.to_numpy()
Y_rideType

In [ ]:
Y = data[['Distance','rideType']]
Y = Y.to_numpy()
Y

## Training Score

In [ ]:
# example of training a final regression model
from sklearn.linear_model import LinearRegression
# fit final model
model = LinearRegression()
model.fit(X[0:160], Y_distance[0:160])
# make a prediction
Xnew = X[160:167]
ynew = model.predict(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
    j = 160
    print("X=%s, Predicted=%s, Actual Distance=%s, Actual Ride Type=%s" % (Xnew[i], ynew[i],Y_distance[j+i],Y_rideType[j+i]))

In [ ]:
result = model.predict([[8,6,1,9,15,0]])
result

## LogisticRegression for RideType

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X, Y_rideType)
result_ridetype = clf.predict([[8,6,1,20,3,0]])
print("Result type prediction=%s" % result_ridetype)

result_ridetype = clf.predict([[8,6,1,10,12,1]])
print("Result type prediction=%s" % result_ridetype)

In [ ]:
#Ride for the weekend
result_ridetype = clf.predict([[8,6,1,8,12,3],[9,7,1,20,17,3]])
print("Result type prediction=%s" % result_ridetype)

#Ride for Weekend
result = model.predict([[9,6,1,10,12,3],[9,7,1,20,17,3]])
print(result)

### Export Model as a file

In [ ]:
import pickle
# Save to file in the model folder
distance_model_file = "model/distance_model.pkl"
with open(distance_model_file, 'wb') as file:
    pickle.dump(model, file)
    
ridetype_model_file = "model/ridetype_model.pkl"
with open(ridetype_model_file, 'wb') as file:
    pickle.dump(clf, file)